最初の10分間の対戦データからPCゲームの勝敗予測しよう！

- 精度評価は、評価関数「Accuracy」を使用します。
- 評価値は0～1の値をとり、精度が高いほど大きな値となります。
- 今回運営の設定するTier変更の閾値はAccuracy=0.785とします。

In [1]:
import numpy as np
import pandas as pd

In [2]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
sub = pd.read_csv('data/sample_submit.csv')

In [3]:
train

,gameId,blueFirstBlood,blueKills,blueDeaths,blueAssists,blueEliteMonsters,blueDragons,blueTotalGold,blueTotalExperience,blueWins
0,0,0,5,8,6,0,0,14536,17256,0
1,1,1,10,1,5,0,0,14536,17863,0
2,2,0,3,10,2,0,0,17409,17256,0
3,3,1,7,10,8,0,0,19558,18201,0
4,4,0,4,9,4,0,0,17409,17256,0
...,...,...,...,...,...,...,...,...,...,...
7995,9993,1,9,6,14,0,0,18513,18201,1
7996,9994,0,3,10,4,0,0,17381,19797,0
7997,9995,1,9,3,9,2,1,18274,18491,1
7998,9997,0,5,10,6,1,1,18274,18491,1


In [4]:
test

,gameId,blueFirstBlood,blueKills,blueDeaths,blueAssists,blueEliteMonsters,blueDragons,blueTotalGold,blueTotalExperience
0,9,0,7,6,6,0,0,16961,18201
1,15,0,6,6,6,2,1,18513,18021
2,18,1,6,4,3,0,0,13475,17256
3,23,0,5,4,7,0,0,17409,17256
4,31,0,10,8,9,0,0,18117,18472
...,...,...,...,...,...,...,...,...,...
1995,9971,0,6,3,7,0,0,17409,17256
1996,9980,0,4,4,4,0,0,17409,17256
1997,9983,1,6,3,3,1,1,18513,18201
1998,9996,1,10,9,9,1,1,18513,18201


In [5]:
sub

,9,0
0,15,0
1,18,0
2,23,0
3,31,0
4,32,0
...,...,...
1994,9971,0
1995,9980,0
1996,9983,0
1997,9996,0


In [6]:
train.isnull().sum()

gameId                 0
blueFirstBlood         0
blueKills              0
blueDeaths             0
blueAssists            0
blueEliteMonsters      0
blueDragons            0
blueTotalGold          0
blueTotalExperience    0
blueWins               0
dtype: int64

In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 10 columns):
gameId                 8000 non-null int64
blueFirstBlood         8000 non-null int64
blueKills              8000 non-null int64
blueDeaths             8000 non-null int64
blueAssists            8000 non-null int64
blueEliteMonsters      8000 non-null int64
blueDragons            8000 non-null int64
blueTotalGold          8000 non-null int64
blueTotalExperience    8000 non-null int64
blueWins               8000 non-null int64
dtypes: int64(10)
memory usage: 625.1 KB


In [8]:
test.isnull().sum()

gameId                 0
blueFirstBlood         0
blueKills              0
blueDeaths             0
blueAssists            0
blueEliteMonsters      0
blueDragons            0
blueTotalGold          0
blueTotalExperience    0
dtype: int64

In [9]:
# 学習データを特徴量と目的変数に分ける
train_x = train.drop(['blueWins'], axis=1)
train_y = train['blueWins']

# テストデータは特徴量のみなので、そのままでよい
test_x = test.copy()

In [10]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 9 columns):
gameId                 2000 non-null int64
blueFirstBlood         2000 non-null int64
blueKills              2000 non-null int64
blueDeaths             2000 non-null int64
blueAssists            2000 non-null int64
blueEliteMonsters      2000 non-null int64
blueDragons            2000 non-null int64
blueTotalGold          2000 non-null int64
blueTotalExperience    2000 non-null int64
dtypes: int64(9)
memory usage: 140.8 KB


# 特徴量作成

In [11]:
# 変数gameIdを除外する
train_x = train_x.drop(['gameId'], axis=1)
test_x = test_x.drop(['gameId'], axis=1)

In [12]:
from sklearn.preprocessing import OneHotEncoder

# 元データをコピーする
train_x = train.drop(['blueWins'], axis=1).copy()
test_x = test.copy()

# 変数gameIdを除外する
train_x = train_x.drop(['gameId'], axis=1)
test_x = test_x.drop(['gameId'], axis=1)

# one-hot encodingを行う
cat_cols = ['blueFirstBlood', 'blueEliteMonsters', 'blueDragons']
ohe = OneHotEncoder(categories='auto', sparse=False)
ohe.fit(train_x[cat_cols].fillna('NA'))

# one-hot encodingのダミー変数の列名を作成する
ohe_columns = []
for i, c in enumerate(cat_cols):
    ohe_columns += [f'{c}_{v}' for v in ohe.categories_[i]]

# one-hot encodingによる変換を行う
ohe_train_x = pd.DataFrame(ohe.transform(train_x[cat_cols].fillna('NA')), columns=ohe_columns)
ohe_test_x = pd.DataFrame(ohe.transform(test_x[cat_cols].fillna('NA')), columns=ohe_columns)

# one-hot encoding済みの変数を除外する
train_x = train_x.drop(cat_cols, axis=1)
test_x = test_x.drop(cat_cols, axis=1)

# one-hot encodingで変換された変数を結合する
train_x = pd.concat([train_x, ohe_train_x], axis=1)
test_x = pd.concat([test_x, ohe_test_x], axis=1)

# 数値変数の欠損値を学習データの平均で埋める
num_cols = ['blueKills', 'blueDeaths', 'blueAssists', 'blueTotalGold', 'blueTotalExperience']
for col in num_cols:
    train_x[col].fillna(train_x[col].mean(), inplace=True)
    test_x[col].fillna(train_x[col].mean(), inplace=True)

# 対数変換する
train_x['blueTotalGold'] = np.log1p(train_x['blueTotalGold'])
train_x['blueTotalExperience'] = np.log1p(train_x['blueTotalExperience'])
test_x['blueTotalGold'] = np.log1p(test_x['blueTotalGold'])
test_x['blueTotalExperience'] = np.log1p(test_x['blueTotalExperience'])

In [13]:
train_x

,blueKills,blueDeaths,blueAssists,blueTotalGold,blueTotalExperience,blueFirstBlood_0,blueFirstBlood_1,blueEliteMonsters_0,blueEliteMonsters_1,blueEliteMonsters_2,blueDragons_0,blueDragons_1
0,5,8,6,9.584452,9.755973,1.0,0.0,1.0,0.0,0.0,1.0,0.0
1,10,1,5,9.584452,9.790543,0.0,1.0,1.0,0.0,0.0,1.0,0.0
2,3,10,2,9.764800,9.755973,1.0,0.0,1.0,0.0,0.0,1.0,0.0
3,7,10,8,9.881191,9.809287,0.0,1.0,1.0,0.0,0.0,1.0,0.0
4,4,9,4,9.764800,9.755973,1.0,0.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
7995,9,6,14,9.826282,9.809287,0.0,1.0,1.0,0.0,0.0,1.0,0.0
7996,3,10,4,9.763190,9.893336,1.0,0.0,1.0,0.0,0.0,1.0,0.0
7997,9,3,9,9.813289,9.825093,0.0,1.0,0.0,0.0,1.0,0.0,1.0
7998,5,10,6,9.813289,9.825093,1.0,0.0,0.0,1.0,0.0,0.0,1.0


# モデル作成

In [14]:
from xgboost import XGBClassifier
# xgb = XGBClassifier(n_estimators=20, random_state=71)

from sklearn.linear_model import LogisticRegression
# lr = LogisticRegression(solver='lbfgs', max_iter=300)

from sklearn import neighbors
# knn = neighbors.KNeighborsClassifier(5, weights = 'distance')

from sklearn.ensemble import RandomForestClassifier
# rf = RandomForestClassifier()

In [15]:
# !pip install keras
# !pip install tensorflow

In [16]:
# NN
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Activation

# nn = Sequential()
# nn.add(Dense(30, input_dim=train_x.shape[1]))
# nn.add(Activation('relu'))
# nn.add(Dense(10))
# nn.add(Activation('softmax'))

# nn.compile(loss='categorical_crossentropy',
#               optimizer='adam',
#               metrics=['accuracy'])

# Y_train = to_categorical(train_y, 10)

# hist = nn.fit(train_x, Y_train,
#                  epochs=30, batch_size=100,verbose=0)

# バリデーション

In [17]:
#それぞれの分類精度を調べる

from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold

In [18]:
#xgb

# 各foldのスコアを保存するリスト
scores_accuracy = []

# クロスバリデーションを行う
# 学習データを4つに分割し、うち1つをバリデーションデータとすることを、バリデーションデータを変えて繰り返す
kf = KFold(n_splits=4, shuffle=True, random_state=71)
for tr_idx, va_idx in kf.split(train_x):
    # 学習データを学習データとバリデーションデータに分ける
    tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
    tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

    # モデルの学習を行う
    model = XGBClassifier(n_estimators=20, random_state=71)
    model.fit(tr_x, tr_y)

    # バリデーションデータの予測値を確率で出力する
    va_pred = model.predict_proba(va_x)[:, 1]

    # バリデーションデータでのスコアを計算する
    accuracy = accuracy_score(va_y, va_pred > 0.5)

    # そのfoldのスコアを保存する
    scores_accuracy.append(accuracy)

# 各foldのスコアの平均を出力する
accuracy = np.mean(scores_accuracy)
print(f'accuracy: {accuracy:.4f}')

accuracy: 0.7616


In [19]:
#lr

# 各foldのスコアを保存するリスト
scores_accuracy = []

# クロスバリデーションを行う
# 学習データを4つに分割し、うち1つをバリデーションデータとすることを、バリデーションデータを変えて繰り返す
kf = KFold(n_splits=4, shuffle=True, random_state=71)
for tr_idx, va_idx in kf.split(train_x):
    # 学習データを学習データとバリデーションデータに分ける
    tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
    tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

    # モデルの学習を行う
    model = LogisticRegression(solver='lbfgs', max_iter=300)
    model.fit(train_x, train_y)
    pred = model.predict_proba(test_x)[:, 1]

    # バリデーションデータでのスコアを計算する
    accuracy = accuracy_score(va_y, va_pred > 0.5)

    # そのfoldのスコアを保存する
    scores_accuracy.append(accuracy)

# 各foldのスコアの平均を出力する
accuracy = np.mean(scores_accuracy)
print(f'accuracy: {accuracy:.4f}')

/Users/kumagaimayuko/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Users/kumagaimayuko/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


accuracy: 0.5542


/Users/kumagaimayuko/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
/Users/kumagaimayuko/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


In [20]:
#knn

# 各foldのスコアを保存するリスト
scores_accuracy = []

# クロスバリデーションを行う
# 学習データを4つに分割し、うち1つをバリデーションデータとすることを、バリデーションデータを変えて繰り返す
kf = KFold(n_splits=4, shuffle=True, random_state=71)
for tr_idx, va_idx in kf.split(train_x):
    # 学習データを学習データとバリデーションデータに分ける
    tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
    tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

    # モデルの学習を行う
    model = neighbors.KNeighborsClassifier(5, weights = 'distance')
    model.fit(tr_x, tr_y)

    # バリデーションデータの予測値を確率で出力する
    va_pred = model.predict(va_x)

    # バリデーションデータでのスコアを計算する
    accuracy = accuracy_score(va_y, va_pred > 0.5)

    # そのfoldのスコアを保存する
    scores_accuracy.append(accuracy)

# 各foldのスコアの平均を出力する
accuracy = np.mean(scores_accuracy)
print(f'accuracy: {accuracy:.4f}')

accuracy: 0.6744


In [21]:
#rf

# 各foldのスコアを保存するリスト
scores_accuracy = []

# クロスバリデーションを行う
# 学習データを4つに分割し、うち1つをバリデーションデータとすることを、バリデーションデータを変えて繰り返す
kf = KFold(n_splits=4, shuffle=True, random_state=71)
for tr_idx, va_idx in kf.split(train_x):
    # 学習データを学習データとバリデーションデータに分ける
    tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
    tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

    # モデルの学習を行う
    rf = RandomForestClassifier()
    rf.fit(tr_x, tr_y)

    # バリデーションデータの予測値を確率で出力する
    va_pred = rf.predict_proba(va_x)[:, 1]

    # バリデーションデータでのスコアを計算する
    accuracy = accuracy_score(va_y, va_pred > 0.5)

    # そのfoldのスコアを保存する
    scores_accuracy.append(accuracy)

# 各foldのスコアの平均を出力する
accuracy = np.mean(scores_accuracy)
print(f'accuracy: {accuracy:.4f}')

accuracy: 0.7474


/Users/kumagaimayuko/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/kumagaimayuko/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/kumagaimayuko/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/kumagaimayuko/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarni

In [22]:
import matplotlib.pyplot as plt

fti = rf.feature_importances_

print('Feature Importances:')
for i, feat in enumerate(train_x.columns):
    print('\t{0:20s} : {1:>.6f}'.format(feat, fti[i]))

Feature Importances:
	blueKills            : 0.141796
	blueDeaths           : 0.150250
	blueAssists          : 0.159167
	blueTotalGold        : 0.193831
	blueTotalExperience  : 0.269986
	blueFirstBlood_0     : 0.012553
	blueFirstBlood_1     : 0.012993
	blueEliteMonsters_0  : 0.010680
	blueEliteMonsters_1  : 0.010544
	blueEliteMonsters_2  : 0.004819
	blueDragons_0        : 0.014561
	blueDragons_1        : 0.018821


In [ ]:
#nn

# 各foldのスコアを保存するリスト
scores_accuracy = []

# クロスバリデーションを行う
# 学習データを4つに分割し、うち1つをバリデーションデータとすることを、バリデーションデータを変えて繰り返す
kf = KFold(n_splits=4, shuffle=True, random_state=71)
for tr_idx, va_idx in kf.split(train_x):
    # 学習データを学習データとバリデーションデータに分ける
    tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
    tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

    # モデルの学習を行う
    nn = Sequential()
    nn.add(Dense(30, input_dim=train_x.shape[1]))
    nn.add(Activation('relu'))
    nn.add(Dense(10))
    nn.add(Activation('softmax'))

    nn.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    Y_train = to_categorical(train_y, 10)

    hist = nn.fit(train_x, Y_train,
                     epochs=30, batch_size=100,verbose=0)

    # バリデーションデータの予測値を確率で出力する
    va_pred = nn.predict_classes(va_x, batch_size=1)

    # バリデーションデータでのスコアを計算する
    accuracy = accuracy_score(va_y, va_pred > 0.5)

    # そのfoldのスコアを保存する
    scores_accuracy.append(accuracy)

# 各foldのスコアの平均を出力する
accuracy = np.mean(scores_accuracy)
print(f'accuracy: {accuracy:.4f}')

# モデルチューニング

In [ ]:
import itertools

# チューニング候補とするパラメータを準備する
param_space = {
    'max_depth': [3, 5, 7, 9, 11, 13],
    'min_child_weight': [1.0, 2.0, 4.0, 8.0, 12.0, 16.0]
}

# 探索するハイパーパラメータの組み合わせ
param_combinations = itertools.product(param_space['max_depth'], param_space['min_child_weight'])

# 各パラメータの組み合わせ、それに対するスコアを保存するリスト
params = []
scores = []

# 各パラメータの組み合わせごとに、クロスバリデーションで評価を行う
for max_depth, min_child_weight in param_combinations:

    score_folds = []
    # クロスバリデーションを行う
    # 学習データを4つに分割し、うち1つをバリデーションデータとすることを、バリデーションデータを変えて繰り返す
    kf = KFold(n_splits=4, shuffle=True, random_state=123456)
    for tr_idx, va_idx in kf.split(train_x):
        # 学習データを学習データとバリデーションデータに分ける
        tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
        tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

        # モデルの学習を行う
        model = XGBClassifier(n_estimators=20, random_state=71,
                              max_depth=max_depth, min_child_weight=min_child_weight)
        model.fit(tr_x, tr_y)

        # バリデーションデータでのスコアを計算し、保存する
        va_pred = model.predict_proba(va_x)[:, 1]
        accuracy = accuracy_score(va_y, va_pred > 0.5)
        score_folds.append(accuracy)

    # 各foldのスコアを平均する
    score_mean = np.mean(score_folds)

    # パラメータの組み合わせ、それに対するスコアを保存する
    params.append((max_depth, min_child_weight))
    scores.append(score_mean)

# 最もスコアが良いものをベストなパラメータとする
best_idx = np.argsort(scores)[0]
best_param = params[best_idx]
print(f'max_depth: {best_param[0]}, min_child_weight: {best_param[1]}')

In [ ]:
# 各foldのスコアを保存するリスト
scores_accuracy = []

# クロスバリデーションを行う
# 学習データを4つに分割し、うち1つをバリデーションデータとすることを、バリデーションデータを変えて繰り返す
kf = KFold(n_splits=4, shuffle=True, random_state=71)
for tr_idx, va_idx in kf.split(train_x):
    # 学習データを学習データとバリデーションデータに分ける
    tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
    tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

    # モデルの学習を行う
    model = XGBClassifier(n_estimators=20, random_state=71,
                              max_depth=3, min_child_weight=12.0)
    model.fit(tr_x, tr_y)

    # バリデーションデータの予測値を確率で出力する
    va_pred = model.predict_proba(va_x)[:, 1]

    # バリデーションデータでのスコアを計算する
    accuracy = accuracy_score(va_y, va_pred > 0.5)

    # そのfoldのスコアを保存する
    scores_accuracy.append(accuracy)

# 各foldのスコアの平均を出力する
accuracy = np.mean(scores_accuracy)
print(f'accuracy: {accuracy:.4f}')

In [ ]:
model = XGBClassifier(n_estimators=20, random_state=71,
                              max_depth=3, min_child_weight=12.0)

model.fit(train_x, train_y)

# テストデータの予測値を確率で出力する
pred = model.predict_proba(test_x)[:, 1]

# テストデータの予測値を二値に変換する
pred_label = np.where(pred > 0.5, 1, 0)

# 提出用ファイルの作成
submission = pd.DataFrame({'gameId': test['gameId'], 'blueWins': pred_label})
submission.to_csv('output/sub.csv',header=False,index=False)
pd.read_csv('output/sub.csv')

In [ ]:
sub

# ロジスティック回帰用の特徴量の作成

In [ ]:
# -----------------------------------
# アンサンブル
# -----------------------------------
from sklearn.linear_model import LogisticRegression

# xgboostモデル
model_xgb = XGBClassifier(n_estimators=20, random_state=71)
model_xgb.fit(train_x, train_y)
pred_xgb = model_xgb.predict_proba(test_x)[:, 1]

# ロジスティック回帰モデル
# xgboostモデルとは異なる特徴量を入れる必要があるので、別途train_x2, test_x2を作成した
model_lr = LogisticRegression(solver='lbfgs', max_iter=300)
model_lr.fit(train_x2, train_y)
pred_lr = model_lr.predict_proba(test_x2)[:, 1]

# 予測値の加重平均をとる
pred = pred_xgb * 0.8 + pred_lr * 0.2
pred_label = np.where(pred > 0.5, 1, 0)

In [ ]:
submission_ens = pd.DataFrame({'gameId': test['gameId'], 'blueWins': pred_label})
submission_ens.to_csv('output/sub_ens.csv',header=False,index=False)
pd.read_csv('output/sub_ens.csv')

In [ ]:
sub

In [ ]:
np.arange(0,1,0.1)[1:]

In [ ]:
import itertools

# チューニング候補とするパラメータを準備する
param_space = {
    'pred_xgb_weight': np.arange(0,1,0.1)[1:],
    'pred_lr_weight': np.arange(0,1,0.1)[1:]
}

# 探索するハイパーパラメータの組み合わせ
param_combinations = itertools.product(param_space['pred_xgb_weight'], param_space['pred_lr_weight'])

# 各パラメータの組み合わせ、それに対するスコアを保存するリスト
params = []
scores = []

# 各パラメータの組み合わせごとに、クロスバリデーションで評価を行う
for pred_xgb_weight, pred_lr_weight in param_combinations:

    score_folds = []
    # モデルの学習を行う
    mmodel = XGBClassifier(n_estimators=20, random_state=71,
                           max_depth=3, min_child_weight=12)
    model_xgb.fit(tr_x, tr_y)
    pred_xgb = model_xgb.predict_proba(test_x)[:, 1]

    # ロジスティック回帰モデル
    # xgboostモデルとは異なる特徴量を入れる必要があるので、別途train_x2, test_x2を作成した
    model_lr = LogisticRegression(solver='lbfgs', max_iter=300)
    model_lr.fit(tr_x, tr_y)
    pred_lr = model_lr.predict_proba(test_x2)[:, 1]

    # 予測値の加重平均をとる
    pred = pred_xgb * pred_xgb_weight + pred_lr * pred_lr_weight
    pred_label = np.where(pred > 0.5, 1, 0)

    # バリデーションデータでのスコアを計算し、保存する
    va_pred = model.predict_proba(va_x)[:, 1]
    accuracy = accuracy_score(va_y, va_pred > 0.5)
    score_folds.append(accuracy)

    # 各foldのスコアを平均する
    score_mean = np.mean(score_folds)

    # パラメータの組み合わせ、それに対するスコアを保存する
    params.append((max_depth, min_child_weight))
    scores.append(score_mean)

# 最もスコアが良いものをベストなパラメータとする
best_idx = np.argsort(scores)[0]
best_param = params[best_idx]
print(f'pred_xgb_weight: {best_param[0]}, pred_lr_weight: {best_param[1]}')

In [ ]:
len(scores)

In [ ]:
# ロジスティック回帰のみ
pred_label_lr = np.where(pred_lr > 0.5, 1, 0)
submission_lr = pd.DataFrame({'gameId': test['gameId'], 'blueWins': pred_label_lr})
submission_lr.to_csv('output/sub_lr.csv',header=False,index=False)
pd.read_csv('output/sub_lr.csv')